In [78]:
import numpy as np
import os
from PIL import Image,ImageOps
from sklearn.model_selection import train_test_split
import random
from keras.models import Sequential,Model
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Input, BatchNormalization
from keras.optimizers import Adam,legacy
import pandas as pd

first gathering data from celebA

first step : gather data and process it

In [24]:
def load_images_from_folder(folder, size=(256, 256), sample_size=1000):
    images = []
    all_filenames = os.listdir(folder)
    sampled_filenames = random.sample(all_filenames, sample_size)

    for filename in sampled_filenames:
        img = Image.open(os.path.join(folder, filename)).convert('RGB')
        img = ImageOps.fit(img, size, Image.Resampling.LANCZOS)
        images.append(np.array(img))
    return images


def preprocess_images(images):
    images = np.array(images, dtype=np.float32) / 255.0
    return images

In [25]:
folder_path = './archive/img_align_celeba/img_align_celeba' 

images = load_images_from_folder(folder_path)
images = preprocess_images(images)
print(np.array(images.shape))
#split test and train images
train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)


[1000  256  256    3]


second step: set up a GAN using keras

In [26]:
def build_generator(z_dim, initial_img_size=(8, 8, 256)):
    model = Sequential()
    model.add(Dense(np.prod(initial_img_size), input_dim=z_dim))
    model.add(Reshape(initial_img_size))

    # Calculate the number of upsampling steps needed
    num_upsamples = int(np.log2(256 / initial_img_size[0]))  # Assuming final size is 256x256

    # Upsampling blocks
    num_filters = 128
    for _ in range(num_upsamples - 1):
        model.add(Conv2DTranspose(num_filters, kernel_size=4, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.01))
        num_filters //= 2  # Halving the filters after each upsampling

    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))

    return model

def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.01))

    current_size = min(image_shape[0], image_shape[1])
    while current_size > 8:  
        model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.01))
        current_size //= 2

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model


step 3: create and set up a GAN model

In [27]:
# GAN parameters
image_shape = (256, 256, 3)
z_dim = 100

# Build and compile the discriminator
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build the generator
generator = build_generator(z_dim, initial_img_size=(8, 8, 256))
# Keep discriminator's parameters constant for generator training
discriminator.trainable = False

# Build and compile the GAN 
gan = Sequential([generator, discriminator])
gan.compile(loss='binary_crossentropy', optimizer=Adam())

step 4: training the GAN model

In [28]:
def save_images(epoch, generator, output_dir="generated_images", n_images=10, z_dim=100, image_size=256):
    noise = np.random.normal(0, 1, (n_images, z_dim))
    gen_images = generator.predict(noise)

    gen_images = 0.5 * gen_images + 0.5
    gen_images = (gen_images * 255).astype(np.uint8)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, img in enumerate(gen_images):
        if img.shape[-1] == 1:
            img = np.squeeze(img, axis=-1)
            img = np.stack((img,) * 3, axis=-1)
        img = Image.fromarray(img)
        img.save(f"{output_dir}/epoch_{epoch}_img_{i}.png")
        
def train_gan(generator, discriminator, gan, epochs, batch_size, train_images,save_interval=1,method='gan'):
    half_batch = int(batch_size / 2)
    metrics_list = []
#out put 
    output_dir = 'generated_images_after_{}_epochs_{}'.format(epochs,method)
    log_name = 'log_after_{}_epochs'.format(epochs)
    
    with open(log_name, 'a') as log_file:
        for epoch in range(epochs):
            
            # Select a random half batch of real images
            idx = np.random.randint(0, train_images.shape[0], half_batch)
            real_imgs = train_images[idx]
            noise = np.random.normal(0, 1, (half_batch, z_dim))
            gen_imgs = generator.predict(noise)
    
            d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
            
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            valid_y = np.array([1] * batch_size)
    
            # Train the generator
            g_loss = gan.train_on_batch(noise, valid_y)
            #log the process
            metrics_list.append({
                'Epoch': epoch,
                'Discriminator Loss': d_loss[0],
                'Generator Loss': g_loss
            })
            log_message = f"Epoch: {epoch}, discriminator loss: {d_loss[0]}, Generator loss: {g_loss}\n"
            log_file.write(log_message)
            log_file.flush()
            if epoch % save_interval == 0:
                save_images(epoch, generator,output_dir)
        metrics_df = pd.DataFrame(metrics_list)
        metrics_df.to_csv(f'training_metrics_after_{epochs}_epochs_{method}.csv', index=False)

In [29]:
train_gan(generator, discriminator, gan, epochs=5000, batch_size=32, train_images=train_images, save_interval=50)

1/1 [==============================] - 0s 95ms/step


step 5: set up fully connected GAN (FCGAN)

In [62]:
def build_fc_generator(z_dim, img_shape):
    flattened_size = np.prod(img_shape)
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim, activation='relu'))
    model.add(BatchNormalization())  # Adding batch normalization
    model.add(Dense(flattened_size, activation='tanh'))  # Change to 'tanh' if images are scaled to [-1, 1]
    model.add(Reshape(img_shape))
    return model


def build_fc_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model



In [63]:
z_dim = 100
image_shape = (256,256,3)
fc_generator = build_fc_generator(z_dim, image_shape)
fc_discriminator = build_fc_discriminator(image_shape)

# Compile the discriminator
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)  # Commonly used values in DCGANs
fc_discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


fc_discriminator.trainable = False

# Create the GAN model
gan_input = Input(shape=(z_dim,))
fake_image = fc_generator(gan_input)
gan_output = fc_discriminator(fake_image)
fc_gan = Model(gan_input, gan_output)

# Compile the GAN model
fc_gan_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
fc_gan.compile(loss='binary_crossentropy', optimizer=fc_gan_optimizer)

step 6: train the model

In [74]:
def save_images(epoch, generator, output_dir, n_images, z_dim, img_shape):
    noise = np.random.normal(0, 1, (n_images, z_dim))
    gen_images = generator.predict(noise)

    gen_images = (gen_images * 255).astype(np.uint8)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, img in enumerate(gen_images):
        
        if img.ndim == 2:
            img = np.stack((img,)*3, axis=-1)  
        elif img.shape[-1] == 1:
            img = np.squeeze(img, axis=-1)
            img = np.stack((img,)*3, axis=-1)  

        img = Image.fromarray(img, 'RGB')
        img.save(os.path.join(output_dir, f"epoch_{epoch}_image_{i}.png"))


def train_fc_gan(generator, discriminator, gan, epochs, batch_size, train_images, z_dim, save_interval,method='fcgan'):
    half_batch = int(batch_size / 2)
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))
    g_labels = np.ones((batch_size, 1))
    output_dir_images = f'generated_images_after_{epochs}_epochs_{method}'
    metrics_list = []
    for epoch in range(epochs):
       
        # Select a random batch of real images
        idx = np.random.randint(0, train_images.shape[0], half_batch)
        real_imgs = train_images[idx]

        # Generate a batch of new fake images
        noise = np.random.normal(0, 1, (half_batch, z_dim))
        fake_imgs = generator.predict(noise)

        # Train the discriminator (real classified as ones and fakes as zeros)
        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        g_loss = gan.train_on_batch(noise, g_labels)
        metrics_list.append({
            'Epoch': epoch,
            'Discriminator Loss': d_loss[0],
            'Generator Loss': g_loss
        })
       
        if epoch % save_interval == 0:
            save_images(epoch, generator, output_dir_images, 10, z_dim, image_shape)
            print(f'Epoch: {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')

    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv(f'training_metrics_after_{epochs}_epochs_{method}.csv', index=False)


In [65]:
train_fc_gan(fc_generator, fc_discriminator, fc_gan, epochs=1000, batch_size=32, train_images=train_images, save_interval=50,z_dim=100)


1/1 [==============================] - 0s 29ms/step
Epoch: 0, Discriminator Loss: [0.69446135 0.3125    ], Generator Loss: 0.6928171515464783
1/1 [==============================] - 0s 31ms/step
Epoch: 50, Discriminator Loss: [2.45684576 0.5       ], Generator Loss: 0.061239246279001236
1/1 [==============================] - 0s 32ms/step
Epoch: 100, Discriminator Loss: [6.47167253 0.5       ], Generator Loss: 0.07101905345916748
1/1 [==============================] - 0s 30ms/step
Epoch: 150, Discriminator Loss: [2.42463589 0.53125   ], Generator Loss: 0.3384436368942261
1/1 [==============================] - 0s 31ms/step
Epoch: 200, Discriminator Loss: [8.0718565 0.53125  ], Generator Loss: 0.4566129446029663
1/1 [==============================] - 0s 30ms/step
Epoch: 250, Discriminator Loss: [13.36901855  0.5       ], Generator Loss: 0.2072720229625702
1/1 [==============================] - 0s 29ms/step
Epoch: 300, Discriminator Loss: [4.20427656 0.53125   ], Generator Loss: 0.476944744

step 7: set up dcgan

In [88]:
def build_dcgan_generator(z_dim):
    model = Sequential()

    
    initial_img_size = (8, 8, 256)  
    dense_size = np.prod(initial_img_size)  

    model.add(Dense(dense_size, input_dim=z_dim))
    model.add(Reshape(initial_img_size))

    model.add(Conv2DTranspose(256, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))

    return model


def build_dcgan_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

step 8: train the dcgan

In [92]:
z_dim = 100
image_shape = (256, 256, 3)

dcgan_generator = build_dcgan_generator(z_dim)
dcgan_discriminator = build_dcgan_discriminator(image_shape)


optimizer = legacy.Adam(learning_rate=0.0002, beta_1=0.5)
dcgan_discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

dcgan_discriminator.trainable = False
gan_input = Input(shape=(z_dim,))
gan_output = dcgan_discriminator(dcgan_generator(gan_input))
dcgan = Model(gan_input, gan_output)

# Compile the GAN
dcgan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [94]:
train_fc_gan(dcgan_generator, dcgan_discriminator, dcgan, epochs=5000, batch_size=32, train_images=train_images, save_interval=50,z_dim=100,method='dcgan')


1/1 [==============================] - 0s 141ms/step
Epoch: 0, Discriminator Loss: [3.21506697e-04 1.00000000e+00], Generator Loss: 0.003871020395308733
1/1 [==============================] - 0s 141ms/step
Epoch: 50, Discriminator Loss: [0.00234833 1.        ], Generator Loss: 0.002044376917183399
1/1 [==============================] - 0s 140ms/step
Epoch: 100, Discriminator Loss: [0.00196121 1.        ], Generator Loss: 1.0013010978582315e-05
1/1 [==============================] - 0s 141ms/step
Epoch: 150, Discriminator Loss: [5.54284439e-04 1.00000000e+00], Generator Loss: 0.000664645223878324
1/1 [==============================] - 0s 148ms/step
Epoch: 200, Discriminator Loss: [8.37815278e-04 1.00000000e+00], Generator Loss: 0.05516452714800835
1/1 [==============================] - 0s 144ms/step
Epoch: 250, Discriminator Loss: [7.88602374e-04 1.00000000e+00], Generator Loss: 0.0013303142040967941
1/1 [==============================] - 0s 160ms/step
Epoch: 300, Discriminator Loss: [8